# PyTorch: Optimization & Training

https://github.com/pytorch/examples/tree/master/mnist

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Preprocess

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

# Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization

- Model과 Optimization 설정

In [6]:
model = Net().to(device) # device에 담아줌

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

- Parameters들 확인
    - Weight, Bais를 순서대로 보여줌

In [8]:
params = list(model.parameters())
for i in range(8): # 꼭 봐야하는 건 아닌데 파라미터들 궁금하니까..
    print(params[i].size())
    # conv1의 weight
    # conv1의 bais
    # conv2의 weight
    # conv2의 bais
    # fc1
    # 
    
# tensorflow의 model.summary()기능 대신해서..

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [9]:
model.train() # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [10]:
data, target = next(iter(train_loader)) # 첫번째 데이터만 load

In [11]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [12]:
data, target = data.to(device), target.to(device)

In [13]:
data.shape, target.shape
# 겉으로는 변환없는데 data를 어디에다가 탑재해주겠다 compile 해준 것

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [14]:
# optimizer 설정해주기전에 깨끗하게 clear해줘야해
optimizer.zero_grad()

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [15]:
output = model(data)

- Model에서 예측한 결과를 Loss Function에 넣음
    - 여기 예제에서는 Negative Log-Likelihood Loss 라는 Loss Function을 사용

In [16]:
# output(prediction)냈으면 얼마나 틀렸는지 봐야겠지
# target과 비교
loss = F.nll_loss(output, target)

- Back Propagation을 통해 Gradients를 계산

In [17]:
loss.backward()

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [19]:
optimizer.step() # optimizer update

# 여기까지 학습 1-step

# Start Training

위의 최적화 과정을 반복하여 학습 시작

In [20]:
epochs = 1
log_interval = 100 # 몇번 step마다 log볼 것인지 설정

In [21]:
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = data.to(device), target.to(device) # 1. 데이터 뽑음
        optimizer.zero_grad() # 2. gradient clear
        output = model(data) # 3. model에 input 넣기
        loss = F.nll_loss(output, target) # 4.output과 target 비교하여 loss function 얻음
        loss.backward() # 5. 얻은 loss를 backpropagation해서 기울기 계산
        optimizer.step () # 6. 얻은 기울기를 optimizer에 update
        
        if batch_idx % log_interval == 0: # 중간중간 log 확인해서 상황 보기
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset), # len(train_loader.dataset): 전체 데이터 수 반환
            # 전체 데이터 중 몇 번째 데이터 인지
            100 * batch_idx / len(train_loader), loss.item()
            # 전체 데이터 중 몇 %인지               # loss의 값 출력
            ))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290262
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.228430
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.119932
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.891091
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.566622
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.210287
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.991650
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.777674
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.732145
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.624069


# Evaluation

- 앞에서 **model.train()** 모드로 변한 것처럼 평가 할 때는 **model.eval()**로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [22]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- torch.no_grad() : autograd engine, 즉 backpropagation이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [23]:
test_loss = 0
correct = 0

with torch.no_grad(): # Test Mode는 torch.no_grad() 해주는 게 중요
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    # 이번엔 optimizer update용이 아니라 계산용도
    test_loss += F.nll_loss(output, target, reduction='sum').item() # reduction='sum': batch_size별이 아니라 한번에 하나의 스칼라로 return
    
    pred = output.argmax(dim=1, keepdim=True) # keepdim=True:차원 수 유지
    correct = pred.eq(target.view_as(pred)).sum()
    # pred.eq(target.view_as(pred)):같은지 안 같은지 True/False
    # .sum().item():True끼리 summation한 다음, item을 correct에 저장

In [29]:
output.shape

torch.Size([64, 10])

In [28]:
pred.shape # keepdim으로 차원 수(위의 output shape)는 유지한 채로 argmax 

torch.Size([64, 1])

In [36]:
pred

tensor([[5],
        [0],
        [2],
        [6],
        [2],
        [8],
        [5],
        [3],
        [1],
        [9],
        [7],
        [7],
        [0],
        [4],
        [0],
        [1],
        [1],
        [1],
        [1],
        [2],
        [4],
        [1],
        [3],
        [3],
        [2],
        [4],
        [0],
        [7],
        [1],
        [1],
        [0],
        [8],
        [2],
        [4],
        [3],
        [6],
        [7],
        [9],
        [6],
        [8],
        [2],
        [6],
        [3],
        [9],
        [1],
        [2],
        [1],
        [8],
        [2],
        [3],
        [0],
        [8],
        [7],
        [9],
        [7],
        [1],
        [4],
        [9],
        [3],
        [9],
        [1],
        [9],
        [6],
        [9]])

In [30]:
target.shape

torch.Size([64])

In [32]:
target.view_as(pred).shape

torch.Size([64, 1])

In [35]:
target

tensor([5, 0, 2, 8, 9, 8, 5, 3, 1, 4, 7, 8, 0, 4, 0, 1, 1, 1, 4, 2, 4, 1, 3, 3,
        2, 4, 0, 7, 8, 1, 0, 8, 2, 4, 8, 6, 9, 3, 6, 8, 2, 6, 3, 9, 1, 1, 1, 8,
        2, 5, 0, 8, 7, 9, 7, 1, 4, 9, 3, 9, 1, 9, 6, 9])

In [37]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [38]:
pred.eq(target.view_as(pred)).sum() # True의 개수

tensor(53)

In [39]:
pred.eq(target.view_as(pred)).sum().item()

53

In [24]:
test_loss # pred와 target이 얼마나 같은지

33.71516036987305

In [25]:
correct

tensor(53)

In [41]:
test_loss /= len(test_loader.dataset) # test_loss의 평균

In [42]:
test_loss

0.003371516036987305

In [44]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

# log 표현
print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))


Test set: Average Loss: 0.4794, Accuracy: 8662/10000 (87%)



In [ ]:
# epoch 돌 때마다 Train Mode에서 학습하고, Test Mode에서 테스트

for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = data.to(device), target.to(device) # 1. 데이터 뽑음
        optimizer.zero_grad() # 2. gradient clear
        output = model(data) # 3. model에 input 넣기
        loss = F.nll_loss(output, target) # 4.output과 target 비교하여 loss function 얻음
        loss.backward() # 5. 얻은 loss를 backpropagation해서 기울기 계산
        optimizer.step () # 6. 얻은 기울기를 optimizer에 update
        
        if batch_idx % log_interval == 0: # 중간중간 log 확인해서 상황 보기
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset), # len(train_loader.dataset): 전체 데이터 수 반환
            # 전체 데이터 중 몇 번째 데이터 인지
            100 * batch_idx / len(train_loader), loss.item()
            # 전체 데이터 중 몇 %인지               # loss의 값 출력
            ))
    
    # Test Mode
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # log 표현
    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))